## Run #3 - Full formulation

In [1]:
# dependencies

import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from pulp import *

In [10]:
listSolvers()

['GLPK_CMD',
 'PYGLPK',
 'CPLEX_CMD',
 'CPLEX_PY',
 'GUROBI',
 'GUROBI_CMD',
 'MOSEK',
 'XPRESS',
 'XPRESS',
 'XPRESS_PY',
 'PULP_CBC_CMD',
 'COIN_CMD',
 'COINMP_DLL',
 'CHOCO_CMD',
 'MIPCL_CMD',
 'SCIP_CMD',
 'HiGHS_CMD']

Econ/env gains

In [9]:
# data inputs
# load time series data
data = pd.read_csv('./all-series.csv')
data = data.drop(data.columns[168:])

data = data.set_index('Hour', drop=True)
irradiance = data['Irradiance (Wh/m2)']          # Wh/m2
sma = data['Consumption Sma (kWh/h)'] * 1.2              # time series with electricity consumption
saft = data['Consumption Saft (kWh/h)'] * 1.05
oe_cons = data['DH plant Consumption (kWh/h)'] * 1.05
turb_prod = data['Turbine Production (kWh/h)']

cons = sma + saft + oe_cons

hourly_price = data['Day Ahead (EUR/MWh)'] / 1000      # time series with electricity spot price

fcrn_vol = data['FCR-N (MW)'] * 1000
fcrn_price = data['FCR-N (EUR/MW)'] / 1000
fcrup_vol = data['FCR-D up (MW)'] * 1000
fcrup_price = data['FCR-D up (EUR/MW)'] / 1000
fcrdown_vol = data['FCR-D down (MW)'] * 1000
fcrdown_price = data['FCR-D down (EUR/MW)'] / 1000

# network tariff
net_charge = 0.04257    # eur/kWh

# pv parameters
sqm_pv = 30000          # m2
conv_eff = 0.2          # %
price_pv = 120          # eur / sqm

# battery parameters
batt_cap = 1000          # kWh
batt_pwr = 500          # kW
deg_price = 0.011       # eur/kWh
price_batt = 320        # eur / kWh

# turbine parameters
turb_proportion = 0.05  # % of turbine production available for consumption
turb_opex = 0.03583     # eur/kWh

# fcr parameters
fcr_share = 0.02        # share of overall se-4 fcr served

# co2 data
co2grid = 0.039         # kgco2/kwh 
co2pv = 350             # kgco2/m2
co2bess = 89            # kgco2/kwh
lcpv = 30               # years
lcbess = 20             # years

CAPEX = price_pv * sqm_pv + price_batt * batt_cap

# displacements to be run:
#   - Mar 21st, 2022 - init_disp = 385
#   - Jun 20th, 2022 - init_disp = 476
#   - Sep 19th, 2022 - init_disp = 567
#   - Dec 19th, 2022 - init_disp = 658
init_disp = 0 # no of days from march 1st, 2021 initial time step is desired

init = 0 + init_disp * 24
n = 8760

# data for optimization
load = np.array(cons[init:init+n])
PV_gen = np.array(irradiance[init:init+n] * (sqm_pv / 1000) * conv_eff) 
price = np.array(hourly_price[init:init+n])

# create prob
prob = LpProblem('GridPVBatt', LpMinimize)

# dec vars
# energy from grid
E_G = [LpVariable('E_G_{}'.format(i), 0, None) for i in range(n)]
# energy from PV
E_PV = [LpVariable('E_PV_{}'.format(i), 0, None) for i in range(n)]
# energy curtailed
E_curt = [LpVariable('E_curt_{}'.format(i), 0, None) for i in range(n)]
# energy charged
E_ch_pv = [LpVariable('E_ch_pv_{}'.format(i), 0, batt_pwr) for i in range(n)]
E_ch_grid = [LpVariable('E_ch_grid_{}'.format(i), 0, batt_pwr) for i in range(n)]
E_ch_fcrn = [LpVariable('E_ch_fcrn_{}'.format(i), 0, batt_pwr) for i in range(n)]
E_ch_fcrup = [LpVariable('E_ch_fcrup_{}'.format(i), 0, batt_pwr) for i in range(n)]
# energy discharged
E_dis_cons = [LpVariable('E_dis_cons_{}'.format(i), 0, batt_pwr) for i in range(n)]
E_dis_sold = [LpVariable('E_dis_sold_{}'.format(i), 0, batt_pwr) for i in range(n)]
E_dis_fcrn = [LpVariable('E_dis_fcrn_{}'.format(i), 0, batt_pwr) for i in range(n)]
E_dis_fcrdown = [LpVariable('E_dis_fcrdown_{}'.format(i), 0, batt_pwr) for i in range(n)]
prob += E_dis_cons[0] == 0
prob += E_dis_sold[0] == 0
prob += E_dis_fcrn[0] == 0
prob += E_dis_fcrdown[0] == 0
batt_status = [LpVariable('batt_status_{}'.format(i), cat='Binary') for i in range(n)]
M = 1000000     # large positive number that acts as upper bound
# energy from turbine
E_turb_cons = [LpVariable('E_turb_cons_{}'.format(i), 0, None) for i in range(n)]
# soc
SOC = [LpVariable('SOC_{}'.format(i), 0, batt_cap) for i in range(n)]
prob += SOC[0] == 0

# obj function: min total elec cost 
# - assume price = 0 for pv electricity
# - assume price = deg_price per kWh charge/discharged
# - assume price of curtailment = spot price (not realistic)
# - assume price of sold energy from batt = spot price
prob += lpSum([  E_G[t] *           (price[t] + net_charge)
               + E_PV[t] *          (0)
               + E_curt[t] *        (-price[t] + net_charge) 
               + E_ch_pv[t] *       (deg_price)
               + E_ch_grid[t] *     (deg_price + net_charge + price[t])
               + E_ch_fcrn[t] *     (deg_price - fcrn_price[t])
               + E_ch_fcrup[t] *    (deg_price - fcrup_price[t])
               + E_dis_cons[t] *    (deg_price)
               + E_dis_sold[t] *    (deg_price + net_charge - price[t])
               + E_dis_fcrn[t] *    (deg_price - fcrn_price[t])
               + E_dis_fcrdown[t] * (deg_price - fcrdown_price[t]) 
               + E_turb_cons[t] *   (turb_opex * 1.5 + net_charge)] 
               for t in range(n))

# constraints
for t in range(1, n):
    # soc
    prob += SOC[t] == SOC[t-1] + E_ch_grid[t] + E_ch_pv[t] + E_ch_fcrn[t] + E_ch_fcrup[t] - E_dis_cons[t] - E_dis_sold[t] - E_dis_fcrdown[t] - E_dis_fcrn[t]

for t in range(0, n):

    # power balance
    prob += E_G[t] + E_PV[t] + E_dis_cons[t] + E_dis_sold[t] + E_dis_fcrdown[t] + E_dis_fcrn[t] + E_turb_cons[t] == load[t] + E_ch_grid[t] + E_ch_pv[t] + E_ch_fcrn[t] + E_ch_fcrup[t] + E_curt[t]

    # pv cons always lower than prod
    prob += E_PV[t] + E_curt[t] + E_ch_pv[t] == PV_gen[t]

    # charge and discharge limits
    prob += E_dis_sold[t] + E_dis_cons[t] + E_dis_fcrdown[t] + E_dis_fcrn[t] + E_ch_pv[t] + E_ch_fcrn[t] + E_ch_fcrup[t] + E_ch_grid[t] <= batt_pwr
    prob += E_ch_pv[t] + E_ch_grid[t] + E_ch_fcrn[t] + E_ch_fcrup[t] <= M * batt_status[t]
    prob += E_dis_cons[t] + E_dis_sold[t] + E_dis_fcrn[t] + E_dis_fcrdown[t] <= M * (1 - batt_status[t])

    # turbine limits
    prob += E_turb_cons[t] <= turb_prod[t] * turb_proportion

    # fcr limits
    prob += E_ch_fcrn[t] + E_dis_fcrn[t] <= fcr_share * fcrn_vol[t]
    prob += E_ch_fcrup[t] <= fcr_share * fcrup_vol[t]
    prob += E_dis_fcrdown[t] <= fcr_share * fcrdown_vol[t]

# solve
solver = pulp.GUROBI()
prob.solve(solver)
# print(LpStatus[status])

# make records to prep df
res = []
for t in range(n):
    record = {  'Hour': t,
                'Load': load[t],
                'PV production': PV_gen[t],
                'Spot price': price[t],
                'Energy from grid': E_G[t].varValue,
                'Energy from PV': E_PV[t].varValue,
                'Energy curtailed': E_curt[t].varValue,
                'Energy charged from PV': E_ch_pv[t].varValue,
                'Energy charged from grid': E_ch_grid[t].varValue,
                'Energy charged for FCR-N': E_ch_fcrn[t].varValue,
                'Energy charged for FCR-up': E_ch_fcrup[t].varValue,
                'Energy consumed from battery': E_dis_cons[t].varValue,
                'Energy sold from battery': E_dis_sold[t].varValue,
                'Energy discharged for FCR-N': E_dis_fcrn[t].varValue,
                'Energy discharged for FCR-down': E_dis_fcrdown[t].varValue,
                'Energy consumed from turbine': E_turb_cons[t].varValue,
                'SOC': SOC[t].varValue
    }
    res.append(record)

df = pd.DataFrame.from_records(res)
df.set_index('Hour', inplace=True)
df = df.round(2)
# print(df.to_string())
df.to_csv('00-full-results.csv', encoding='utf-8')

emissions = ((sum(df['Energy from PV'])*2 + sum(df['Energy consumed from battery'])*2) 
             * co2grid - sqm_pv * (2*co2pv/lcpv) - batt_cap * (2*co2bess/lcbess))

print(f'~~~~~~~~~~~~~~~\nInstallation summary')
print(f'Installed PV (square meters): {sqm_pv : 0.1f}')
print(f'Battery capacity (kWh): {batt_cap : 0.1f}')
print(f'Battery power (kW): {batt_pwr : 0.1f}')
print(f'CAPEX (EUR): {CAPEX : 0.1f}\n~~~~~~~~~~~~~~~')
print(f'Minimum cost of electricity: {pulp.value(prob.objective)*2 : 0.1f}€')
print(f'Emissions avoided (kgCO2 eq): {emissions : 0.1f}')

# df.plot(figsize=[15,5])
# plt.show()

AttributeError: module 'pulp.pulp' has no attribute 'GUROBI'

Representative weeks

In [ ]:
# data inputs
# load time series data
data = pd.read_csv('./all-series.csv')
data = data.drop(data.columns[168:])

data = data.set_index('Hour', drop=True)
irradiance = data['Irradiance (Wh/m2)']          # Wh/m2
sma = data['Consumption Sma (kWh/h)'] * 1.2              # time series with electricity consumption
saft = data['Consumption Saft (kWh/h)'] * 1.05
oe_cons = data['DH plant Consumption (kWh/h)'] * 1.05
turb_prod = data['Turbine Production (kWh/h)']

cons = sma + saft + oe_cons

hourly_price = data['Day Ahead (EUR/MWh)'] / 1000      # time series with electricity spot price

fcrn_vol = data['FCR-N (MW)'] * 1000
fcrn_price = data['FCR-N (EUR/MW)'] / 1000
fcrup_vol = data['FCR-D up (MW)'] * 1000
fcrup_price = data['FCR-D up (EUR/MW)'] / 1000
fcrdown_vol = data['FCR-D down (MW)'] * 1000
fcrdown_price = data['FCR-D down (EUR/MW)'] / 1000

# network tariff
net_charge = 0.04257    # eur/kWh

# pv parameters
sqm_pv = 30000          # m2
conv_eff = 0.2          # %
price_pv = 120          # eur / sqm

# battery parameters
batt_cap = 1000          # kWh
batt_pwr = 500          # kW
deg_price = 0.011       # eur/kWh
price_batt = 320        # eur / kWh

# turbine parameters
turb_proportion = 0.05  # % of turbine production available for consumption
turb_opex = 0.03583     # eur/kWh

# fcr parameters
fcr_share = 0.02        # share of overall se-4 fcr served

# co2 data
co2grid = 0.039         # kgco2/kwh 
co2pv = 350             # kgco2/m2
co2bess = 89            # kgco2/kwh
lcpv = 30               # years
lcbess = 20             # years

CAPEX = price_pv * sqm_pv + price_batt * batt_cap

# displacements to be run:
#   - Mar 21st, 2022 - init_disp = 385
#   - Jun 20th, 2022 - init_disp = 476
#   - Sep 19th, 2022 - init_disp = 567
#   - Dec 19th, 2022 - init_disp = 658
init_disp = 658 # no of days from march 1st, 2021 initial time step is desired

init = 0 + init_disp * 24
n = 168

# data for optimization
load = np.array(cons[init:init+n])
PV_gen = np.array(irradiance[init:init+n] * (sqm_pv / 1000) * conv_eff) 
price = np.array(hourly_price[init:init+n])

# create prob
prob = LpProblem('GridPVBatt', LpMinimize)

# dec vars
# energy from grid
E_G = [LpVariable('E_G_{}'.format(i), 0, None) for i in range(n)]
# energy from PV
E_PV = [LpVariable('E_PV_{}'.format(i), 0, None) for i in range(n)]
# energy curtailed
E_curt = [LpVariable('E_curt_{}'.format(i), 0, None) for i in range(n)]
# energy charged
E_ch_pv = [LpVariable('E_ch_pv_{}'.format(i), 0, batt_pwr) for i in range(n)]
E_ch_grid = [LpVariable('E_ch_grid_{}'.format(i), 0, batt_pwr) for i in range(n)]
E_ch_fcrn = [LpVariable('E_ch_fcrn_{}'.format(i), 0, batt_pwr) for i in range(n)]
E_ch_fcrup = [LpVariable('E_ch_fcrup_{}'.format(i), 0, batt_pwr) for i in range(n)]
# energy discharged
E_dis_cons = [LpVariable('E_dis_cons_{}'.format(i), 0, batt_pwr) for i in range(n)]
E_dis_sold = [LpVariable('E_dis_sold_{}'.format(i), 0, batt_pwr) for i in range(n)]
E_dis_fcrn = [LpVariable('E_dis_fcrn_{}'.format(i), 0, batt_pwr) for i in range(n)]
E_dis_fcrdown = [LpVariable('E_dis_fcrdown_{}'.format(i), 0, batt_pwr) for i in range(n)]
prob += E_dis_cons[0] == 0
prob += E_dis_sold[0] == 0
prob += E_dis_fcrn[0] == 0
prob += E_dis_fcrdown[0] == 0
batt_status = [LpVariable('batt_status_{}'.format(i), cat='Binary') for i in range(n)]
M = 1000000     # large positive number that acts as upper bound
# energy from turbine
E_turb_cons = [LpVariable('E_turb_cons_{}'.format(i), 0, None) for i in range(n)]
# soc
SOC = [LpVariable('SOC_{}'.format(i), 0, batt_cap) for i in range(n)]
prob += SOC[0] == 0

# obj function: min total elec cost 
# - assume price = 0 for pv electricity
# - assume price = deg_price per kWh charge/discharged
# - assume price of curtailment = spot price (not realistic)
# - assume price of sold energy from batt = spot price
prob += lpSum([  E_G[t] *           (price[t] + net_charge)
               + E_PV[t] *          (0)
               + E_curt[t] *        (-price[t] + net_charge) 
               + E_ch_pv[t] *       (deg_price)
               + E_ch_grid[t] *     (deg_price + net_charge + price[t])
               + E_ch_fcrn[t] *     (deg_price - fcrn_price[t])
               + E_ch_fcrup[t] *    (deg_price - fcrup_price[t])
               + E_dis_cons[t] *    (deg_price)
               + E_dis_sold[t] *    (deg_price + net_charge - price[t])
               + E_dis_fcrn[t] *    (deg_price - fcrn_price[t])
               + E_dis_fcrdown[t] * (deg_price - fcrdown_price[t]) 
               + E_turb_cons[t] *   (turb_opex * 1.5 + net_charge)] 
               for t in range(n))

# constraints
for t in range(1, n):
    # soc
    prob += SOC[t] == SOC[t-1] + E_ch_grid[t] + E_ch_pv[t] + E_ch_fcrn[t] + E_ch_fcrup[t] - E_dis_cons[t] - E_dis_sold[t] - E_dis_fcrdown[t] - E_dis_fcrn[t]

for t in range(0, n):

    # power balance
    prob += E_G[t] + E_PV[t] + E_dis_cons[t] + E_dis_sold[t] + E_dis_fcrdown[t] + E_dis_fcrn[t] + E_turb_cons[t] == load[t] + E_ch_grid[t] + E_ch_pv[t] + E_ch_fcrn[t] + E_ch_fcrup[t] + E_curt[t]

    # pv cons always lower than prod
    prob += E_PV[t] + E_curt[t] + E_ch_pv[t] == PV_gen[t]

    # charge and discharge limits
    prob += E_dis_sold[t] + E_dis_cons[t] + E_dis_fcrdown[t] + E_dis_fcrn[t] + E_ch_pv[t] + E_ch_fcrn[t] + E_ch_fcrup[t] + E_ch_grid[t] <= batt_pwr
    prob += E_ch_pv[t] + E_ch_grid[t] + E_ch_fcrn[t] + E_ch_fcrup[t] <= M * batt_status[t]
    prob += E_dis_cons[t] + E_dis_sold[t] + E_dis_fcrn[t] + E_dis_fcrdown[t] <= M * (1 - batt_status[t])

    # turbine limits
    prob += E_turb_cons[t] <= turb_prod[t] * turb_proportion

    # fcr limits
    prob += E_ch_fcrn[t] + E_dis_fcrn[t] <= fcr_share * fcrn_vol[t]
    prob += E_ch_fcrup[t] <= fcr_share * fcrup_vol[t]
    prob += E_dis_fcrdown[t] <= fcr_share * fcrdown_vol[t]

# solve
status = prob.solve()
print(LpStatus[status])

# energy balance area plot for one week

areas = []
for t in range(n):
    record = {  'Hour': t,
                'Demand': load[t],
                'Grid cons': E_G[t].varValue,
                'PV cons': E_PV[t].varValue,
                'Battery cons': E_dis_cons[t].varValue,
                'Turbine cons': E_turb_cons[t].varValue,
                'Curtailment': -E_curt[t].varValue,
                'Charge PV': -E_ch_pv[t].varValue,
                'Charge grid': -E_ch_grid[t].varValue,
                'Battery sales': E_dis_sold[t].varValue,
                'FCR-N (charge)': -E_ch_fcrn[t].varValue,
                'FCR-N (discharge)': E_dis_fcrn[t].varValue,
                'FCR-up': -E_ch_fcrup[t].varValue,
                'FCR-down': E_dis_fcrdown[t].varValue,
    }
    areas.append(record)

df = pd.DataFrame.from_records(areas)
df.set_index('Hour', inplace=True)
df = df.round(2)

if init_disp == 385:
    plot_title = 'Mar 21st, 2022'
elif init_disp == 476:
    plot_title = 'Jun 20th, 2022'
elif init_disp == 567:
    plot_title = 'Sep 19th, 2022'
elif init_disp == 658:
    plot_title = 'Dec 19th, 2022'
else:
    plot_title = 'Energy balance'

ax = df.plot.area(y=['Grid cons', 'PV cons', 'Battery cons', 'Turbine cons', 'Battery sales', 'FCR-N (discharge)', 'FCR-down',
                     'Curtailment', 'Charge PV', 'Charge grid', 'FCR-N (charge)', 'FCR-up'], 
                  figsize = (10, 5), stacked=True, alpha=0.3, legend=None)

df.plot(y='Demand', ax=ax, color='red', linestyle='-', legend=None)

plt.title(plot_title)
plt.ylabel('kWh/h')
# plt.legend(bbox_to_anchor=(1, 0.5))

plt.show()
